# Machine Learning II: Assignments 1
Leon Berghoff, Jiawei Li, Strahinja Trenkic, Otto Riess

In [1]:
# Make sure you have installed the required package and restarted the notebook server
# %pip install ipywidgets ipympl

## Covid-19 Disasters
The SIR model is a 3-compartment model. Extend this model to 4 compartments, where the 4th compartment is for deaths $D$. Mortality is modelled by new transitions from $I → D$ deﬁned by the mortality rate $µ$. Susceptible and Recovered do not die.

(a) Derive the corresponding system of equations for $S$, $I$, $R$ and $D$. But this is not the only difference to SIR. In addition, the basic reproduction number may now depend on $\mu$ as well, how?

The SIRD model is:

$$
\begin{aligned}
\frac{d S}{d t} &= -\frac{\beta I S}{N} \\
\frac{d I}{d t} &=\frac{\beta I S}{N}-\gamma I-\mu I \\
\frac{d R}{d t} &=\gamma I \\
\frac{d D}{d t} &=\mu I \\
\end{aligned}
$$

Assume that the birth rates and death rates are equal, the population $N$ is constant:

$$
S + I + R + D = N
$$

The basic reproduction number, $R_0$, is defined as the expected number of cases directly caused by one case at time 0 in a population where all individuals are susceptible. In SIRD model, the $R_0$ is:

$$
R_0 = \frac{\beta}{\gamma + \mu}
$$

(b) Assume that the basic reproduction number $R_{0}$ for B.1.1.7 is not exactly known but only the range $R_{0} \in[3.0,4.0]$. Assume that the mortality rate $\mu$ is also not exactly known but only the range $\mu \in[0.4 \%, 4 \%]$. Study how these parameter uncertainties affect the prediction of $D$ at $t=365 d$. What about the cumulative number of deaths after a year?

In [8]:
%matplotlib inline
import ipywidgets as widgets
from ipywidgets import interactive
import matplotlib.pyplot as plt
import numpy as np
from scipy.integrate import odeint

In [9]:
# A Simple Example
def f(m, b):
    plt.figure(2)
    x = np.linspace(-10, 10, num=1000)
    plt.plot(x, m * x + b)
    plt.ylim(-5, 5)
    plt.show()

interactive_plot = interactive(f, m=(-2.0, 2.0), b=(-3, 3, 0.5))
interactive_plot

interactive(children=(FloatSlider(value=0.0, description='m', max=2.0, min=-2.0), FloatSlider(value=0.0, descr…

In [7]:
# Another example that uses SIRD model
# The output can be further tweaked following the documentation:
# https://ipywidgets.readthedocs.io/en/latest/examples/Using%20Interact.html#More-control-over-the-user-interface:-interactive_output

def plot_sird(r_nought, gamma, mu):
    # Population size
    N = 100
    
    # Initially, only 1 infected
    I0 = 1
    
    # Initially, no recovered individuals, no death
    R0, D0 = 0, 0

    # Initially, S0=N-I0-R0-D0
    S0 = N - I0 - R0 - D0

    # Basic reproduction number r_nought = beta / (gamma + mu)
    beta = r_nought * (gamma + mu)
            
    # SIRD model
    def deriv(y, t, N, beta, gamma, mu):
        S, I, R, D = y
        dSdt = -beta * S * I / N
        dIdt = beta * S * I / N - gamma * I - mu * I
        dRdt = gamma * I
        dDdt = mu * I
        return dSdt, dIdt, dRdt, dDdt

    # Initial conditions vector
    y0 = (S0, I0, R0, D0)
    
    # A grid of time points (in days)
    t = np.linspace(0, 365, 365)
    
    # Integrate the SIRD equations over the time grid t
    ret = odeint(deriv, y0, t, args=(N, beta, gamma, mu))
    S, I, R, D = ret.T
    
    # Plot the data on four separate curves for S(t), I(t), R(t) and D(t)
    fig, ax = plt.subplots(figsize=(10,6), constrained_layout=True)
    fig.suptitle('SIRD Model with Different $R_0$, $\gamma$ and $\mu$', fontsize=14, fontweight='bold')
    ax.set_title(f'$\\beta$={round(beta,2)}, $\\gamma$={round(gamma,2)}, $\\mu$={round(mu,3)}')
    ax.plot(t, S/N, 'b', alpha=0.5, lw=1, label='Susceptible')
    ax.plot(t, I/N, 'r', alpha=0.5, lw=1, label='Infected')
    ax.plot(t, R/N, 'g', alpha=0.5, lw=1, label='Recovered')
    ax.plot(t, D/N, 'k', alpha=0.5, lw=1, label='Deaths')
    ax.set_xlabel('Time / days')
    ax.set_ylabel('Fraction')
    ax.set_ylim(0,1)
    ax.yaxis.set_tick_params(length=0)
    ax.xaxis.set_tick_params(length=0)
    ax.grid(b=True, which='major', c='w', lw=1, ls='-')
    ax.legend()
    return ax

interactive_plot = interactive(plot_sird, r_nought=(3,4,0.1), gamma=(0.01,1,0.01), mu=(0.004,0.04,0.001))
interactive_plot

interactive(children=(FloatSlider(value=3.0, description='r_nought', max=4.0, min=3.0), FloatSlider(value=0.5,…

In [11]:
# The code below is essentially the same as the examples but goes lower level

%matplotlib widget
import ipywidgets as widgets
from ipywidgets import interactive
import matplotlib.pyplot as plt
import numpy as np
from scipy.integrate import odeint

In [12]:
def plot_sird(percent=None, r_nought=None, beta=None, gamma=None, mu=None):
    # Population size
    N = 100
    
    if percent is None:
        # Initially, only 1 infected
        I0 = 1
    else:
        # Start with more than 1% infected individuals
        I0 = percent * N
    
    # Initially, no recovered individuals, no death
    R0, D0 = 0, 0

    # Initially, S0=N-I0-R0-D0
    S0 = N - I0 - R0 - D0

    # Basic reproduction number r_nought = beta / (gamma + mu)
    if beta is None:
        beta = r_nought * (gamma + mu)
    elif gamma is None:
        gamma = beta / r_nought - mu
            
    # SIRD model
    def deriv(y, t, N, beta, gamma, mu):
        S, I, R, D = y
        dSdt = -beta * S * I / N
        dIdt = beta * S * I / N - gamma * I - mu * I
        dRdt = gamma * I
        dDdt = mu * I
        return dSdt, dIdt, dRdt, dDdt

    # Initial conditions vector
    y0 = (S0, I0, R0, D0)
    
    # A grid of time points (in days)
    t = np.linspace(0, 365, 365)
    
    # Integrate the SIRD equations over the time grid t
    ret = odeint(deriv, y0, t, args=(N, beta, gamma, mu))
    S, I, R, D = ret.T
    
    # Plot the data on four separate curves for S(t), I(t), R(t) and D(t)
    ax.set_title(f'$\\beta$={round(beta,2)}, $\\gamma$={round(gamma,2)}, $\\mu$={round(mu,3)}')
    ax.plot(t, S/N, 'b', alpha=0.5, lw=1, label='Susceptible')
    ax.plot(t, I/N, 'r', alpha=0.5, lw=1, label='Infected')
    ax.plot(t, R/N, 'g', alpha=0.5, lw=1, label='Recovered')
    ax.plot(t, D/N, 'k', alpha=0.5, lw=1, label='Deaths')
    ax.legend()
    return ax

In [13]:
output = widgets.Output()

with output:
    fig, ax = plt.subplots(figsize=(6,3), constrained_layout=True)
    fig.suptitle('SIRD Model with Different $R_0$, $\gamma$ and $\mu$', fontsize=14, fontweight='bold')
    # Remove the toolbar and header
    fig.canvas.toolbar_visible = False
    fig.canvas.header_visible = False
    fig.canvas.footer_visible = False
    ax.set_xlabel('Time / days')
    ax.set_ylabel('Fraction')
    ax.set_ylim(0,1)
    ax.yaxis.set_tick_params(length=0)
    ax.xaxis.set_tick_params(length=0)
    ax.grid(b=True, which='major', c='w', lw=1, ls='-')

plot_sird(r_nought=3, gamma=0.01, mu=0.004)

# Create control elements
label_r_nought = widgets.Label(r'$R_0$:', layout=widgets.Layout(width='2em'))
slider_r_nought = widgets.FloatSlider(
    value=3,
    min=3,
    max=4)
label_gamma = widgets.Label(r'$\gamma$:', layout=widgets.Layout(width='2em'))
slider_gamma = widgets.FloatSlider(
    value=0.01,
    min=0.01,
    max=1,
    step=0.01)
label_mu = widgets.Label(r'$\mu$:', layout=widgets.Layout(width='2em'))
slider_mu = widgets.FloatSlider(
    value=0.004,
    min=0.004,
    max=0.0401,
    step=0.001,
    readout_format='.3f')

# Callback functions
def update_r_nought(slider):
    ax.lines = []
    plot_sird(r_nought=slider.new, gamma=slider_gamma.value, mu=slider_mu.value)

def update_gamma(slider):
    ax.lines = []
    plot_sird(r_nought=slider_r_nought.value, gamma=slider.new, mu=slider_mu.value)

def update_mu(slider):
    ax.lines = []
    plot_sird(r_nought=slider_r_nought.value, gamma=slider_gamma.value, mu=slider.new)

# Observe change
slider_r_nought.observe(update_r_nought, 'value')
slider_gamma.observe(update_gamma, 'value')
slider_mu.observe(update_mu, 'value')

controls =  widgets.VBox([widgets.HBox([label_r_nought, slider_r_nought]),
            widgets.HBox([label_gamma, slider_gamma]),
            widgets.HBox([label_mu, slider_mu])])

display(widgets.VBox([output, controls]))

By tweaking the parameters in the plot above, we can see that a higher $R_0$ leads to higher number of deaths. Also, a higher death rate leads to higher number of deaths, which was quite intuitive.

As we are progressing from $R_0$ $= 3$ to $R_0$ $= 4$, we can dinamically notice a steepening in the curves of Infected, Recovered and Died. 

As for the cumulative number of dead at 365t we notice that around 20% of the population dies before we reach saturation at $R_0= 3$. This percentage climbs to around 25% at $R_0 = 4$

(c) Study numerically the effects of a hard versus soft lockdown (by two for you reasonable values of $\beta)$, in terms of $D(365 d)$. What about the cumulative number of deaths after a year? Assume $\mu=1 \%$ and a $\gamma$ compatible with $R_{0}=4$.

In [8]:
output = widgets.Output()

with output:
    fig, ax = plt.subplots(figsize=(6,3), constrained_layout=True)
    fig.suptitle('SIRD Model with Different Lockdown Measure', fontsize=14, fontweight='bold')
    ax.set_xlabel('Time / days')
    ax.set_ylabel('Fraction')
    ax.set_ylim(0,1)
    ax.yaxis.set_tick_params(length=0)
    ax.xaxis.set_tick_params(length=0)
    ax.grid(b=True, which='major', c='w', lw=1, ls='-')
    # Remove the toolbar and header
    fig.canvas.toolbar_visible = False
    fig.canvas.header_visible = False
    fig.canvas.footer_visible = False

plot_sird(r_nought=4, beta=0.1, mu=0.01)

# Create control elements
label_beta = widgets.Label(r'$\beta$:', layout=widgets.Layout(width='2em'))
slider_beta = widgets.FloatSlider(
    value=0.1,
    min=0.1,
    max=10,
    step=0.05)

# Callback functions
def update_beta(slider):
    ax.lines = []
    plot_sird(r_nought=4, beta=slider.new, mu=0.01)

# Observe change
slider_beta.observe(update_beta, 'value')

controls = widgets.HBox([label_beta, slider_beta])

display(widgets.VBox([output, controls]))

By fixing $R_0$ at 4 and $\mu$ at 0.01, we can see that a hard lockdown results in a much lower number of deaths than a soft lockdown.

$$
R_0 = \frac{\beta}{\gamma + \mu}
$$

Numerically this is equal to $R_0$ being harder and harder to push over 1 since the nominator of the fraction becomes smaller and smaller. Similarly, a higher transmision rate, all other factors fixed, will lead to a higher nominator and hence a larger $R_0$.

(b-c) Can you ﬁnd a way to derive and plot the effective reproduction number, $R$, as a function of time, given otherwise ﬁxed parameters?

In [5]:
def plot_r_eff(beta, gamma, mu):
    # Population size
    N = 100

    # Initially, only 1 infected, no recovered individuals, no death
    I0, R0, D0 = 1, 0, 0

    # Initially, S0=N-I0-R0-D0
    S0 = N - I0 - R0 - D0

    # Basic reproduction number R0
    r_nought = beta / (gamma + mu)

    # A grid of time points (in days)
    t = np.linspace(0, 365, 365)

    # SIRD model
    def deriv(y, t, N, beta, gamma, mu):
        S, I, R, D = y
        dSdt = -beta * S * I / N
        dIdt = beta * S * I / N - gamma * I - mu * I
        dRdt = gamma * I
        dDdt = mu * I
        return dSdt, dIdt, dRdt, dDdt

    # Initial conditions vector
    y0 = (S0, I0, R0, D0)

    # Integrate the SIRD equations over the time grid t
    ret = odeint(deriv, y0, t, args=(N, beta, gamma, mu))
    S, I, R, D = ret.T

    # Plot the data on four separate curves for S(t), I(t), R(t) and D(t)
    fig = plt.figure(facecolor='w')
    # Remove the toolbar and header
    fig.canvas.toolbar_visible = False
    fig.canvas.header_visible = False
    fig.canvas.footer_visible = False
    fig.suptitle('The Effective Reproduction Number', fontsize=14, fontweight='bold')

    ax = fig.add_subplot(111, axisbelow=True)
    ax.plot(t, r_nought*S/N, 'gray', alpha=0.5, lw=2, label='R')
    ax.set_title(f'beta={beta}, gamma={gamma}, mu={mu}')
    ax.set_xlabel('Time / days')
    ax.set_ylabel('Fraction')
    ax.set_ylim(0,2)
    ax.yaxis.set_tick_params(length=0)
    ax.xaxis.set_tick_params(length=0)
    ax.grid(b=True, which='major', c='w', lw=2, ls='-')
    legend = ax.legend()
    legend.get_frame().set_alpha(0.5)

In [6]:
plot_r_eff(beta=0.8, gamma=0.3, mu=0.05)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(d) Free choice for the initial conditions $S(0)$ and initial prevalence, $I(0)$. Assume $R(0)=D(0)=0$. If you choose $N=1$, the compartments become fractions of the population number and you can remove $N$ from the entire system of equations. Start with more than $1 \%$ of infected individuals (but not exactly $1 \%)$.

In [10]:
output = widgets.Output()

with output:
    fig, ax = plt.subplots(figsize=(6,3), constrained_layout=True)
    fig.suptitle('SIRD Model with Different Initial Prevalence', fontsize=14, fontweight='bold')
    ax.set_title('beta=0.1, gamma=0.05, mu=0.01')
    ax.set_xlabel('Time / days')
    ax.set_ylabel('Fraction')
    ax.set_ylim(0,1)
    ax.yaxis.set_tick_params(length=0)
    ax.xaxis.set_tick_params(length=0)
    ax.grid(b=True, which='major', c='w', lw=1, ls='-')
    # Remove the toolbar and header
    fig.canvas.toolbar_visible = False
    fig.canvas.header_visible = False
    fig.canvas.footer_visible = False

plot_sird(percent=0.01, beta=0.1, gamma=0.05, mu=0.01)

# Create control elements
label_percent = widgets.Label('Percent:', layout=widgets.Layout(width='4em'))
slider_percent = widgets.FloatSlider(
    value=0.01,
    min=0.01,
    max=1,
    step=0.01)

# Callback functions
def update_percent(slider):
    ax.lines = []
    plot_sird(percent=slider.new, beta=0.1, gamma=0.05, mu=0.01)

# Observe change
slider_percent.observe(update_percent, 'value')

controls = widgets.HBox([label_percent, slider_percent])
display(widgets.VBox([output, controls]))

With a higher initial prevalence, the population reaches its infection peak earlier and the cummulative number of deaths is also larger.